In [1]:
import pandas as pd
import xml.etree.ElementTree as ET

### Analyse and identify traffic/emission heavy junction

In [5]:
def calculate_junction_emissions(xml_file):
    
    '''
    Function to calculate the total emissions for each pollutant at each junction.
    '''
    
    tree = ET.parse(xml_file)
    root = tree.getroot()

    junction_emissions = {}

    for timestep in root.findall('timestep'):
        for vehicle in timestep.findall('vehicle'):
            junction = vehicle.get('lane').split('_')[0]  # Extract junction ID from lane ID
            emissions = {
                'CO2': float(vehicle.get('CO2')),
                'CO': float(vehicle.get('CO')),
                'HC': float(vehicle.get('HC')),
                'NOx': float(vehicle.get('NOx')),
                'PMx': float(vehicle.get('PMx')),
                'fuel': float(vehicle.get('fuel')),
                'noise': float(vehicle.get('noise'))
            }

            if junction in junction_emissions:
                for pollutant in emissions:
                    junction_emissions[junction][pollutant] += emissions[pollutant]
            else:
                junction_emissions[junction] = emissions

    return junction_emissions


In [6]:
junction_emissions = calculate_junction_emissions('../src/data/actuated_output/2023-07-20-21-05-25emission_info_actuated_TL.xml')

In [ ]:
def find_emission_heavy_junctions(junction_emissions, pollutant='CO2', num_junctions=5):
    sorted_junctions = sorted(junction_emissions.items(), key=lambda x: x[1][pollutant], reverse=True)
    return sorted_junctions[:num_junctions]
